In [1]:
using CSV, DataFrames, Statistics, Distributions, StatsBase, InMemoryDatasets
using  Random, Statistics, DataStructures, LinearAlgebra
using HTTP, JSON3, Dates

┌ Warning: Julia started with single thread, to enable multithreaded functionalities in InMemoryDatasets.jl start Julia with multiple threads.
└ @ InMemoryDatasets ~/.julia/packages/InMemoryDatasets/P9NS3/src/InMemoryDatasets.jl:205


In [35]:
function get_daily_data(symbol)

    url1 = "https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol="
    url2 = "&market=USD&apikey=0JA1NVBAL8CHYSCH"
    u = url1 * symbol * url2

    resp = HTTP.request("GET", u)
    rawdata = JSON3.read(String(resp.body))
    information = [i for i in rawdata[Symbol("Time Series (Digital Currency Daily)")]]
    format = DateFormat("y-m-d")
    dates = [Date(String(information[i][1]),format) for i in 1:length(rawdata[Symbol("Time Series (Digital Currency Daily)")])]
    adjclose = [parse(Float64,information[i][2][Symbol("4a. close (USD)")]) for i in 1:length(rawdata[Symbol("Time Series (Digital Currency Daily)")])]
    volume = [parse(Float64,information[i][2][Symbol("5. volume")]) for i in 1:length(rawdata[Symbol("Time Series (Digital Currency Daily)")])]

    returns = [(adjclose[i]-adjclose[i+1])./adjclose[i+1] for i in 1:length(adjclose)-1]
    volume = [(volume[i]-volume[i+1])./volume[i+1] for i in 1:length(volume)-1]

    df = DataFrame(Date = dates[1:end-1], returns = returns, volume = volume)
    col_names = ["Date", "ret_"*symbol, "vol_"*symbol]
    
    DataFrames.rename!(df, col_names)


    return sort!(df,[:Date])
end


get_daily_data (generic function with 1 method)

In [42]:
function combined_data()
    tickers = ["BTC", "ETH"]

    df = DataFrame()
    for j in tickers
        try
            d = get_daily_data(j)
            if isempty(df)
                df = d
            else
                df = sort!(outerjoin(df, d, on=:Date, makeunique=true) , [:Date])
            end
        catch
            println("did not get ", j)
        end
    sleep(rand(2:5))
    end

    odd = [i for i in 1:ncol(df) if i%2==1]
    even = [i for i in 1:ncol(df) if i%2==0]

    dfodd = df[!, odd]
    dfeven = df[!, even]
    df = hcat(dfodd, dfeven)

end

combined_data (generic function with 1 method)

In [43]:
pwd()

"/Users/cpu2/Downloads/qreg_btceth/data"